<a href="https://colab.research.google.com/github/straj-granicy/ML-Stepik/blob/master/210203_Stepik_Ml_contest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

events_train.csv - данные о действиях, которые совершают студенты со стэпами

    step_id - id стэпа
    user_id - анонимизированный id юзера
    timestamp - время наступления события в формате unix date
    action - событие, возможные значения: 

    discovered - пользователь перешел на стэп
    viewed - просмотр шага,
    started_attempt - начало попытки решить шаг, ранее нужно было явно нажать на кнопку - начать решение, перед тем как приступить к решению практического шага
    passed - удачное решение практического шага
    submissions_train.csv - данные о времени и статусах сабмитов к практическим заданиям

    step_id - id стэпа
    timestamp - время отправки решения в формате unix date
    submission_status - статус решения
    user_id - анонимизированный id юзера

In [1]:
import pandas as pd
import numpy as np
% matplotlib inline
import matplotlib as plt
import seaborn as sns

*настройка* графика

In [2]:
sns.set(rc={'figure.figsize':(9,6)})

In [3]:
events_data=pd.read_csv('event_data_train.csv')

обрати внимание - дата записана в timesstamp uniqsdate.возможно хорошо для оптимизации модели. это количество секунд,которые прошли с определенной даты (тут - 1 января 1970)


In [4]:
events_data.head(10)

,step_id,timestamp,action,user_id
0,32815,1434340848,viewed,17632.0
1,32815,1434340848,passed,17632.0
2,32815,1434340848,discovered,17632.0
3,32811,1434340895,discovered,17632.0
4,32811,1434340895,viewed,17632.0
5,32811,1434340895,passed,17632.0
6,32929,1434340921,discovered,17632.0
7,32929,1434340921,viewed,17632.0
8,32929,1434340939,started_attempt,17632.0
9,32929,1434340944,passed,17632.0


timestamp можно сравнивать с значениями - !!

Валидация данных 
Перед тем как начать работу с датасетом, важно убедиться, что он содержит корректные данные и в каких-то столбцах нет откровенно невалидных данных.

Давайте проверим уникальные значения в столбце action.
```
# This is formatted as code
```



In [5]:
events_data.action.unique()

array(['viewed', 'passed', 'discovered', 'started_attempt', 'started_att'],
      dtype=object)

Давайте создадим ещё одну колонку, в которой значение из колонки timestamp будет переведено в формат даты.

Это удобно для группировки по дням, месяцам и т.д

In [6]:
events_data['date'] = pd.to_datetime(events_data.timestamp,unit='s')

In [7]:
events_data.head()

,step_id,timestamp,action,user_id,date
0,32815,1434340848,viewed,17632.0,2015-06-15 04:00:48
1,32815,1434340848,passed,17632.0,2015-06-15 04:00:48
2,32815,1434340848,discovered,17632.0,2015-06-15 04:00:48
3,32811,1434340895,discovered,17632.0,2015-06-15 04:01:35
4,32811,1434340895,viewed,17632.0,2015-06-15 04:01:35


In [10]:
events_data.dtypes

step_id               int64
timestamp             int64
action               object
user_id             float64
date         datetime64[ns]
dtype: object

делаем проверку даты на валидность. начальная дата.

In [8]:
events_data.date.min()

Timestamp('2015-06-15 04:00:48')

In [9]:
events_data.date.max()

Timestamp('2015-12-18 11:35:35')

Работа с временем и датой - очень важный аспект в мире Data Science. Практически все базы данных используют timestamp для определения времени события, в машинном обучении есть целое направление - анализ временных рядов, для аналитики продукта очень важно понимать, как располагаются события во времени.

Подготовил для вас несколько полезных статей:

Работы с датой в Pandas https://medium.com/datadriveninvestor/how-to-work-with-dates-in-pandas-like-a-pro-a84055a4819d и еще один вариант https://www.geeksforgeeks.org/python-working-with-date-and-time-using-pandas/

Работа с датой в Python при помощи модуля datetime


создаем колонку "день"                               dt. date attribute to return the date property of the underlying data of the given Series object.
т.е. из колнки date оно возрващает дату.

In [11]:
events_data['day'] = events_data.date.dt.date

In [12]:
events_data.head()

,step_id,timestamp,action,user_id,date,day
0,32815,1434340848,viewed,17632.0,2015-06-15 04:00:48,2015-06-15
1,32815,1434340848,passed,17632.0,2015-06-15 04:00:48,2015-06-15
2,32815,1434340848,discovered,17632.0,2015-06-15 04:00:48,2015-06-15
3,32811,1434340895,discovered,17632.0,2015-06-15 04:01:35,2015-06-15
4,32811,1434340895,viewed,17632.0,2015-06-15 04:01:35,2015-06-15
